<a href="https://colab.research.google.com/github/masies/CRA/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html
! pip install -q transformers==3.5.0 fast-trees
! git clone -q https://github.com/microsoft/CodeXGLUE.git

from os import walk
import os
import regex

from google.colab import auth
auth.authenticate_user()
project_id = 'helical-loop-303918'
bucket_name = 'code_review_automation'
!gcloud config set project {project_id}



     |██████████████████████████████▉ | 726.3MB 1.2MB/s eta 0:00:22
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/main.py", line 45, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/__init__.py", line 96, in create_command
    module = importlib.import_module(module_path)
  File "/usr/lib/python3.7/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1006, in _gcd_import
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 967, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 677, in _load_unlocked
  File "<frozen importlib._bootstrap_external

In [50]:
!rm -rf ./validation_eval/
!gsutil -m cp -r gs://code_review_automation/fine_tuning/HP_tuning/comment_v2/both/non_pretrained/polynomial/validation_eval .

Copying gs://code_review_automation/fine_tuning/HP_tuning/comment_v2/both/non_pretrained/polynomial/validation_eval/codeANDcomment_code_0_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/comment_v2/both/non_pretrained/polynomial/validation_eval/codeANDcomment_code_100000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/comment_v2/both/non_pretrained/polynomial/validation_eval/codeANDcomment_code_10000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/comment_v2/both/non_pretrained/polynomial/validation_eval/codeANDcomment_code_10200_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/comment_v2/both/non_pretrained/polynomial/validation_eval/codeANDcomment_code_30000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/comment_v2/both/non_pretrained/polynomial/validation_eval/codeANDcomment_code_15300_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/com

In [51]:
def eval(val_folder, tasks):
  for task in tasks:
    print(task)
    _, _, filenames = next(walk(val_folder))
    prediction_paths = [os.path.join(val_folder, f) for f in filenames if f.startswith(task) and f.endswith("predictions")]
    target_path = [os.path.join(val_folder, f) for f in filenames if f.startswith(task) and f.endswith("targets")][0]

    with open(target_path) as f:
        targets = [item.strip() for item in f.readlines()]

    for path in sorted(prediction_paths):
      step = regex.findall( task + "_(\d+)_predictions", path)[0]
      if step.endswith("000"):
        # !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs ./../../../../../{target_path} --hyp ./../../../../../{path} --lang java --params 0.25,0.25,0.25,0.25
        
        with open(path) as f:
          predictions = [item.strip() for item in f.readlines()]
        
        perfect_predictions = 0
        almost_perfect_predictions = 0
        for pred, target in zip(predictions, targets):
          if ' '.join(pred.strip().split()) == ' '.join(target.strip().split()):
            perfect_predictions += 1
          elif ''.join(pred.strip().split()) == ''.join(target.strip().split()):
            almost_perfect_predictions += 1
            index = next((i for i, (char1, char2) in enumerate(zip(pred, target)) if char1 != char2), None)
            
        print("perfect predictions at step " + step + " : ", round(perfect_predictions/len(predictions)*100,2))
        print("almost perfect predictions at step " + step + " : ", round((perfect_predictions + almost_perfect_predictions)/len(predictions)*100,2))
        print()

eval('validation_eval', ["code_code", "codeANDcomment_code"])
# eval('validation_eval', ["code_code"])
      


code_code
perfect predictions at step 100000 :  6.13
almost perfect predictions at step 100000 :  6.3

perfect predictions at step 10000 :  5.37
almost perfect predictions at step 10000 :  5.55

perfect predictions at step 15000 :  5.9
almost perfect predictions at step 15000 :  6.07

perfect predictions at step 20000 :  5.9
almost perfect predictions at step 20000 :  6.01

perfect predictions at step 25000 :  6.25
almost perfect predictions at step 25000 :  6.3

perfect predictions at step 30000 :  6.25
almost perfect predictions at step 30000 :  6.3

perfect predictions at step 35000 :  6.01
almost perfect predictions at step 35000 :  6.19

perfect predictions at step 40000 :  5.78
almost perfect predictions at step 40000 :  5.9

perfect predictions at step 45000 :  6.07
almost perfect predictions at step 45000 :  6.19

perfect predictions at step 50000 :  6.54
almost perfect predictions at step 50000 :  6.6

perfect predictions at step 5000 :  0.0
almost perfect predictions at step 

In [ ]:
def codeBleu(pred, target):
  with open("pred.txt", "w") as f:
    f.write(pred)
  with open("tar.txt", "w") as f:
    f.write(target)
  try :
    a = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs ./../../../../../pred.txt --hyp ./../../../../../tar.txt --lang java --params 0.25,0.25,0.25,0.25
    codeBleu = float(a[1].replace("CodeBLEU score:  ",""))
    print(codeBleu)
  except Exception as e:
    codeBleu = 0